
### Entrada
- x1: Local da Coleta em km
- x2: Nivel da Mare em m
- x3: Umidade do Ar (%)
- x4: Índice Pluvimétrico (mm)¹
- x5: Temperatura (°C)

### Saida
- y1: pH
- y2: Condutividade Elétrica (mS.cmˉ¹)
- y3: Oxigênio Dissolvido (mg.Lˉ¹)
- y4: Sólidos Totais Dissolvidos (ppm)


In [1]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [2]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

In [3]:
import numpy as np
import pandas as pd

def create_dataframe(output, label):
    df = pd.read_excel("../../../data.xlsx")
    x1, x2, x3, x4, x5  = df['x1'], df['x2'], df['x3'], df['x4'], df['x5']

    input = np.vstack([x1, x2, x3, x4, x5 ]).T
    output = np.array(df[output])
    return output, input

output, input = create_dataframe(output="y3", label="oxigenio")

/tmp/ipykernel_161874/160627683.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-03-24 03:03:36.310333: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 03:03:36.371752: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 03:03:36.371795: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 03:03:36.373258: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-24 03:03:36.383190: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 03:03:36.384217: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
    
    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
 
    
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)

          metrics = {
                          'r2': r2,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

In [6]:
from itertools import product
import pickle

class Tester:
  def __init__(self, input, output, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = input, output
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=5,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path) 

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)


  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie): # should be <= to descending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

# Init
  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults(sort_by, header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

In [7]:
tester = Tester(input, output, run_times=25, dataset_run_times=10)

In [8]:
tester.Loop(sort_by='r2',
            boundarie=0.8,
            hidden_sizes = [[5], [10], [15], [20], [25], [30], [35], [40], [45]],
            regularizers=[0.01, 0.1],
            learning_rate=[0.1])

Testando combinacao1: Hidden Size=[5], regularizer=0.01, learning_rate=0.1
+++++++++++ [1_0] | 1 ++++++++++++++++++
Elapsed time:  2.043288019999636
Stopped at epoch:  25
1/1 [==============================] - 0s 26ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  0.6173252950002279
Stopped at epoch:  8
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.5211159689997658
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.39435007600013705
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.4019090120000328
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.38025090100018133
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step
+++++++++++ [1_0] | 7 +++++++++++++

2024-03-24 03:04:38.829986: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


1/1 [==============================] - 0s 62ms/step
+++++++++++ [1_4] | 6 ++++++++++++++++++
Elapsed time:  0.060118739000245114
Stopped at epoch:  0
1/1 [==============================] - 0s 24ms/step
+++++++++++ [1_4] | 7 ++++++++++++++++++
Elapsed time:  0.0496166209995863
Stopped at epoch:  0
1/1 [==============================] - 0s 23ms/step
+++++++++++ [1_4] | 8 ++++++++++++++++++
Elapsed time:  0.08077631500009375
Stopped at epoch:  0
1/1 [==============================] - 0s 26ms/step
+++++++++++ [1_4] | 9 ++++++++++++++++++
Elapsed time:  0.07652537800004211
Stopped at epoch:  0
1/1 [==============================] - 0s 75ms/step
+++++++++++ [1_4] | 10 ++++++++++++++++++
Elapsed time:  0.0976902219999829
Stopped at epoch:  0
1/1 [==============================] - 0s 62ms/step
+++++++++++ [1_4] | 11 ++++++++++++++++++
Elapsed time:  0.07804779100024462
Stopped at epoch:  0
1/1 [==============================] - 0s 34ms/step
+++++++++++ [1_4] | 12 ++++++++++++++++++
Elapsed tim

2024-03-24 03:15:58.074645: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  0.5859224119999453
Stopped at epoch:  6
1/1 [==============================] - 0s 27ms/step
+++++++++++ [6_0] | 21 ++++++++++++++++++
Elapsed time:  0.5835213359996487
Stopped at epoch:  6
1/1 [==============================] - 0s 37ms/step
+++++++++++ [6_0] | 22 ++++++++++++++++++
Elapsed time:  0.787661529000161
Stopped at epoch:  9
1/1 [==============================] - 0s 30ms/step
+++++++++++ [6_0] | 23 ++++++++++++++++++
Elapsed time:  0.629700707000211
Stopped at epoch:  6
1/1 [==============================] - 0s 35ms/step
+++++++++++ [6_0] | 24 ++++++++++++++++++
Elapsed time:  0.8954171509999469
Stopped at epoch:  9
1/1 [==============================] - 0s 31ms/step
+++++++++++ [6_0] | 25 ++++++++++++++++++
Elapsed time:  0.5322945580001033
Stopped at epoch:  6
1/1 [==============================] - 0s 33ms/step
+++++++++++ [6_1] | 1 ++++++++++++++++++
Elapsed time:  0.9775887910000165
Stopped at epoch:  6
1/1 [==============================] - 0s 46ms/step
++

...

DataFrame salvo em ../results/metrics_18_7
+++++++++++ [18_8] | 1 ++++++++++++++++++
Elapsed time:  0.20430623599986575
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [18_8] | 2 ++++++++++++++++++
Elapsed time:  0.4335838289998719
Stopped at epoch:  14
1/1 [==============================] - 0s 15ms/step
+++++++++++ [18_8] | 3 ++++++++++++++++++
Elapsed time:  0.26196327400066366
Stopped at epoch:  8
1/1 [==============================] - 0s 17ms/step
+++++++++++ [18_8] | 4 ++++++++++++++++++
Elapsed time:  0.2393644480007424
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [18_8] | 5 ++++++++++++++++++
Elapsed time:  0.27426385700073297
Stopped at epoch:  9
1/1 [==============================] - 0s 15ms/step
+++++++++++ [18_8] | 6 ++++++++++++++++++
Elapsed time:  0.2834038339997278
Stopped at epoch:  9
1/1 [==============================] - 0s 18ms/step
+++++++++++ [18_8] | 7 ++++++++++++++++++
Elapsed time:  0.

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_18_8_23,0.836157,0.575264,0.887593,0.751537,0.181326,0.520683,0.156005,0.349070
model_18_8_22,0.836523,0.574937,0.890151,0.752575,0.180921,0.521084,0.152455,0.347612
model_18_8_21,0.837293,0.582158,0.887808,0.754821,0.180068,0.512232,0.155707,0.344455
model_18_8_19,0.837824,0.585842,0.887524,0.756391,0.179481,0.507716,0.156101,0.342251
model_18_8_16,0.837961,0.582219,0.892321,0.756947,0.179329,0.512158,0.149443,0.341469
model_18_8_18,0.837985,0.584055,0.890282,0.756848,0.179303,0.509906,0.152273,0.341608
model_18_8_20,0.838028,0.586463,0.887818,0.756814,0.179256,0.506955,0.155694,0.341655
model_18_8_15,0.838157,0.585187,0.890692,0.757561,0.179112,0.508519,0.151705,0.340607
model_18_8_17,0.838513,0.584144,0.893358,0.758319,0.178719,0.509798,0.148004,0.339542
model_18_8_14,0.838533,0.584848,0.893132,0.758539,0.178697,0.508934,0.148319,0.339233


DataFrame salvo em ../results/metrics_18_8
+++++++++++ [18_9] | 1 ++++++++++++++++++
Elapsed time:  0.5962831650003864
Stopped at epoch:  14
1/1 [==============================] - 0s 13ms/step
+++++++++++ [18_9] | 2 ++++++++++++++++++
Elapsed time:  0.24328203200002463
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [18_9] | 3 ++++++++++++++++++
Elapsed time:  0.2827418900005796
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [18_9] | 4 ++++++++++++++++++
Elapsed time:  0.21053966199997376
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [18_9] | 5 ++++++++++++++++++
Elapsed time:  0.18565983299959044
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [18_9] | 6 ++++++++++++++++++
Elapsed time:  0.18985506700028054
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [18_9] | 7 ++++++++++++++++++
Elapsed time:  0

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_18_9_24,0.833395,0.749121,0.541173,0.614702,0.184382,0.171502,0.714207,0.426892
model_18_9_23,0.833921,0.749289,0.543840,0.616521,0.183801,0.171387,0.710055,0.424877
model_18_9_22,0.833939,0.749642,0.543784,0.616598,0.183781,0.171145,0.710143,0.424792
model_18_9_21,0.834647,0.751853,0.546335,0.619006,0.182997,0.169634,0.706172,0.422123
model_18_9_20,0.835479,0.753086,0.550018,0.621844,0.182076,0.168791,0.700439,0.418979
model_18_9_19,0.835712,0.750229,0.552750,0.622718,0.181818,0.170744,0.696186,0.418011
model_18_9_18,0.835949,0.750583,0.553754,0.623499,0.181556,0.170502,0.694623,0.417146
model_18_9_17,0.836187,0.749175,0.555921,0.624470,0.181293,0.171464,0.691251,0.416070
model_18_9_16,0.836416,0.749630,0.556900,0.625266,0.181039,0.171153,0.689727,0.415188
model_18_9_15,0.836454,0.746899,0.558592,0.625492,0.180997,0.173020,0.687093,0.414938


DataFrame salvo em ../results/metrics_18_9
